In [238]:
import numpy as np
from scipy import sparse
from typing import Tuple

def standard_constraints(G, h, A, b)-> Tuple[np.ndarray[np.float64], np.ndarray[np.float64], np.ndarray[np.float64]]:
    """
    Convert constraints Gx <= h and Ax = b into a unified form l <= Ax <= u.

    Parameters:
        G (ndarray): Coefficient matrix for inequality constraints.
        h (ndarray): Upper bound for inequality constraints.
        A (ndarray): Coefficient matrix for equality constraints.
        b (ndarray): Right-hand side for equality constraints.

    Returns:
        A_osqp (csr_matrix): Combined constraint matrix.
        l_osqp (ndarray): Lower bounds.
        u_osqp (ndarray): Upper bounds.
    """
    # Inequality constraints: Gx <= h
    if G is not None and h is not None:
        A_ineq = G
        l_ineq = np.full(h.shape, -np.inf)
        u_ineq = h
    else:
        A_ineq, l_ineq, u_ineq = None, None, None

    # Equality constraints: Ax = b
    if A is not None and b is not None:
        A_eq = A
        l_eq = b
        u_eq = b
    else:
        A_eq, l_eq, u_eq = None, None, None

    # Combine constraints
    A_combined = np.vstack([A_ineq, A_eq]) if A_ineq is not None else sparse.csc_matrix(A_eq)
    l_combined = np.hstack([l_ineq, l_eq]) if l_ineq is not None else l_eq
    u_combined = np.hstack([u_ineq, u_eq]) if u_ineq is not None else u_eq
    return A_combined, l_combined, u_combined



In [239]:
def ADAL(A1:np.ndarray, A2:np.ndarray ,b1:np.ndarray ,b2: np.ndarray,q:np.ndarray,P:np.ndarray,x0, y0, z0, eta, rho, alpha):
    x = x0
    y = y0
    z = z0
    A_osqp,l_osqp,u_osqp = standard_constraints(A2,-b2,A1,-b1)
    def Pc(z):
        for i in range(len(z)):
            if l_osqp[i] <= z[i] and z[i] <= u_osqp[i]:
                pass
            else:
                z[i] = u_osqp[i]
        return z
    def solve_linear_sys():
        zuoshang = P+eta*np.eye(P.shape[0])
        zuoxia = A_osqp
        youshang = A_osqp.T
        youxia = -1/rho*np.eye(A_osqp.shape[0])
        shang = np.block([zuoshang,youshang])
        xia = np.block([zuoxia,youxia])
        M = np.vstack([shang,xia])
        
        b = np.hstack([eta*x-q,z-1/rho*y])
        xv = np.linalg.solve(M, b)
        x_hat,v = np.split(xv,[P.shape[0]])
        return x_hat,v
    
    while(True):
        x_hat,v = solve_linear_sys()
        z_hat = z+1/rho*(v-y)
        x_next = alpha*x_hat + (1-alpha)*x
        z_next = Pc(alpha*z_hat + (1-alpha)*z+1/rho*y)
        y = y+rho*(alpha*z_hat+ (1-alpha)*z-z_next)
        z = z_next
        r = np.linalg.norm(x-x_next)
        x = x_next
        if r<1e-8:
            return x

In [240]:
from qpsolvers import solve_qp

M = np.array([[1.0, 2.0, 0.0], [-8.0, 3.0, 2.0], [0.0, 1.0, 1.0]])
P = M.T @ M  # this is a positive definite matrix
q = np.array([3.0, 2.0, 3.0]) @ M
G = np.array([[1.0, 2.0, 1.0], [2.0, 0.0, 1.0], [-1.0, 2.0, -1.0]])
h = np.array([3.0, 2.0, -2.0])
A = np.array([[1.0, 1.0, 1.0]])
b = np.array([1.0])

x = solve_qp(P, q, G, h, A, b, solver="osqp")
print(f"QP solution: {x = }")

QP solution: x = array([ 0.3076548 , -0.69232615,  1.38448776])


In [241]:
x = ADAL(A,G,-b,-h,q,P,np.array([1.0, 1.0, 1.0]),np.array([1.0, 1.0, 1.0,1.0]),np.array([1.0, 1.0, 1.,1.0]),1,1,1)
print(x)

[ 0.3076923  -0.69230778  1.38461544]
